In [1]:
!nvidia-smi

Mon Jul 13 19:05:57 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  On   | 00000000:65:00.0  On |                  N/A |
| 23%   31C    P8     9W / 250W |    483MiB / 11170MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [4]:
from utils import load_data
from embedding import get_token_data, get_s2v_model, get_p2v_model

In [5]:
train, val, song_meta = load_data()

In [194]:
import pandas as pd
genre_gn_all = pd.read_json('./data/genre_gn_all.json', typ = 'series')
genre = pd.DataFrame(genre_gn_all).reset_index()
genre = genre.rename(columns={"index":"genre_code", 0:"genre_name"})

In [109]:
### genre name list
enre_origin = list(genre["genre_name"])

genre_total = genre_origin
for x in genre_origin:
    if len(x.split(" "))>1:
        genre_total = genre_total + x.split(" ")
    
    if len(x.split("/"))>1:
        genre_total = genre_total + x.split("/")
        
    if len(x.split("-"))>1:
        genre_total = genre_total + x.split("-")
        
    if len(x.split("'"))>1:
        genre_total = genre_total + x.split("'")
        

genre_total = [x for x in genre_total if len(x)!=0]

In [137]:
with open("genre_dic.txt", "w") as f:
    
    for genre in genre_total:
        data = genre+"\t"+"NNG\n"
        f.write(data)

In [256]:
from konlpy.tag import Komoran
komoran = Komoran(userdic="genre_dic.txt")

def get_genre_word_in_title(title):
    title_morph = komoran.pos(title)
    genre_word = [x[0] for x in title_morph if x[0] in genre_total]
    return genre_word

def get_genre_word_in_tag(tags):
    genre_word = [x for x in tags if x in genre_total]
    return genre_word

def get_plylst_genre_by_song(songs):
    
    mapping_list = list(genre["genre_code"])
    plylst_genre = []
    for song in songs:
        tmp = song_meta[song_meta["id"]==song]
        song_genre = list(tmp["song_gn_gnr_basket"])[0] +list(tmp["song_gn_gnr_basket"])[0]
        plylst_genre = plylst_genre+song_genre
        
    plylst_genre = [x for x in plylst_genre if x in mapping_list]
    try:
        plylst_genre_name = [list(genre[genre["genre_code"]==code]["genre_name"])[0] for code in plylst_genre]
    except:
        print(plylst_genre)
        
    return plylst_genre_name

In [ ]:
train["plylst_genre_name"] = train["songs"].map(lambda x : get_plylst_genre_by_song(x))

In [ ]:
val["plylst_genre_name"] = val["songs"].map(lambda x : get_plylst_genre_by_song(x))

In [ ]:
train.to_csv("train_genre_name.csv", encoding="cp949")
val.to_csv("val_genre_name.csv", encoding="cp949")

In [181]:
train["title_to_genre"] = train["plylst_title"].map(lambda x : get_genre_word_in_title(x))
train["tag_to_genre"] = train["tags"].map(lambda x : get_genre_word_in_tag(x))

In [186]:
i=95968
train[i:i+10][["tags", "plylst_title", "title_to_genre", "tag_to_genre"]]

,tags,plylst_title,title_to_genre,tag_to_genre
95968,"[소울, 알앤비]","Cafe Jireh(R&B, 90 , Hip)","[R&B, 90]",[소울]
95969,[기분전환에좋은신나는노래],핫한 요즘 노래,[],[]
95970,"[나였으면, 예뻐보여]",alone,[],[]
95971,"[힙합, 인디음악, 달달, 사랑스러운]",달달한 인디 힙합 음악,[],"[힙합, 인디음악]"
95972,[락],"사랑, 시작되다.",[],[]
95973,[인디],가을의 한 가운데서 듣기 좋은 곡들 : ),[],[]
95974,[락],뜨거운여름쿠바빛낭만,[],[]
95975,[뉴에이지],아름다운 음악을 들으면서 편안한 주말 보내기,[],[뉴에이지]
95976,"[비오는날, 감성, 카페, 로맨틱, 발라드, 드라이브]",비내리는날 카페에서 이노랠들으며 감성을 느껴봐요.,[],[발라드]
95977,"[이디엠, 페스티벌]",EDM 입문을 위한 스테디셀러 리스트! Pt.3,[EDM],[]


In [94]:
genre_total = genre_total + [x.split(" ") for x in genre_total] + [x.split("/") for x in genre_total]

In [62]:
def get_genre_name(song_id):
    genre_list = list(song_meta[song_meta["id"]==song_id]["song_gn_gnr_basket"])[0]
    return [list(genre[genre["genre_code"]==x]["genre_name"])[0] for x in genre_list][0]

In [90]:
[get_genre_name(x) for x in list(train[train["id"]==98228]["songs"])[0]]

['POP',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '록/메탈',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '록/메탈',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 'POP',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '월드뮤직',
 '록/메탈',
 'POP',
 '월드뮤직']

In [87]:
train[80:90]

,tags,id,plylst_title,songs,like_cnt,updt_date
80,"[여행, 드라이브, 산책]",94227,해변가 드라이브,"[84857, 582005, 425148, 517355, 236393, 371945...",6,2015-05-04 14:22:42.000
81,"[힐링, 휴식, 기분전환]",140918,"힐링이 필요하신분 한곡,한곡 천천히 들어보세요","[19172, 403808, 398199, 649626, 493250, 49160,...",7,2016-10-06 02:19:38.000
82,"[분위기좋은, 크리스마스]",60192,느낌있는 크리스마스 쏭,"[187632, 432118, 557519, 423058, 387647, 61771...",1,2019-12-03 20:33:17.000
83,[뉴에이지],95004,★아무때나 듣기좋은 뉴에이지★,"[493304, 433509, 208859, 372058, 95432, 307225...",142,2013-07-29 23:25:40.000
84,"[스트레스, 드라이브]",19726,Indie Rock (Pops),"[347545, 571861, 534576, 576151, 185755, 48877...",4,2014-07-16 11:21:42.000
85,[월드뮤직],41023,정열적인 라틴음악,"[397584, 175393, 172891, 627648, 617009, 21867...",284,2011-10-11 18:30:27.000
86,[기분전환],60834,기분 좋아지는 음악 모음,"[414221, 629700, 647369, 236363, 589691, 84627...",3,2019-01-11 13:13:14.000
87,"[힐링, 휴식, 가을]",20908,가을밤의 풀소리와 함께 듣는 뉴에이지.,"[603070, 489885, 448999, 224568, 696494, 30896...",5,2015-09-05 21:04:36.000
88,[뉴에이지],51433,상쾌한 아침을 여는 싱그러운 뉴에이지 음악,"[148736, 504216, 608545, 200585, 473108, 48547...",1,2015-11-19 11:51:52.000
89,[월드뮤직],98228,공통감각 (Italia Caf'e),"[293412, 682883, 55108, 359440, 62168, 564618,...",24,2015-09-02 09:18:34.000


In [10]:
song_meta["issue_year"] = song_meta["issue_date"].map(lambda x :int(str(x)[:4]))

In [11]:
song_meta

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id,issue_year
0,[GN0901],20140512,불후의 명곡 - 7080 추억의 얄개시대 팝송베스트,2255639,[2727],Feelings,[GN0900],[Various Artists],0,2014
1,"[GN1601, GN1606]",20080421,"Bach : Partitas Nos. 2, 3 & 4",376431,[29966],"Bach : Partita No. 4 In D Major, BWV 828 - II....",[GN1600],[Murray Perahia],1,2008
2,[GN0901],20180518,Hit,4698747,[3361],Solsbury Hill (Remastered 2002),[GN0900],[Peter Gabriel],2,2018
3,"[GN1102, GN1101]",20151016,Feeling Right (Everything Is Nice) (Feat. Popc...,2644882,[838543],Feeling Right (Everything Is Nice) (Feat. Popc...,[GN1100],[Matoma],3,2015
4,"[GN1802, GN1801]",20110824,그남자 그여자,2008470,[560160],그남자 그여자,[GN1800],[Jude Law],4,2011
...,...,...,...,...,...,...,...,...,...,...
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984,1999
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985,1986
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986,2016
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987,2013


In [13]:
train[10:100]

,tags,id,plylst_title,songs,like_cnt,updt_date
10,"[kpop, 댄스, 걸그룹댄스, 스트레스해소]",31804,걸그룹 땐쓰쏭,"[507380, 144826, 553894, 660381, 260730, 28188...",74,2020-04-13 23:36:55.000
11,"[새해, 여행, 프로필음악, 카카오톡, 기분전환, 소원, 프로필, 소망, 다짐, 카톡]",151693,"노래로 의지를 불태우자! ""1일1다짐"" st용 프로필뮤직","[658738, 698779, 124485, 512610, 310028, 17422...",42,2017-02-16 15:45:23.000
12,"[듣고, 우울, 힐링, 이거, 힘내]",112060,지친 너를 위로해줄 제목편지 준비해봤어,"[449808, 691770, 640657, 313395, 697100, 34090...",1,2018-01-04 22:49:29.000
13,"[힙합, 느낌있는, 밤, 새벽, RnB, 감각적인, 드라이브, 국내, 그루브한]",89809,트렌디하고 그루브한 힙합/알앤비 MUSIC,"[525152, 38765, 66139, 696379, 397438, 144461,...",1112,2018-01-09 14:01:53.000
14,"[가을, 재즈]",131092,Autumn in Jazz,"[37194, 493012, 516717, 458409, 573367, 5056, ...",146,2017-08-25 22:38:40.000
...,...,...,...,...,...,...
95,"[꿀, 보이스, 뺏고싶다, 인디, 성대]",34226,하나 들으면 다른 곡도 듣게 되는 마성의 목소리 모음,"[169999, 128184, 87907, 165412, 112310, 707517...",4,2017-02-05 17:29:58.000
96,[발라드],34198,안구를 촉촉히 적셔줄 노래들,"[470207, 654757, 279532, 78541, 117595, 46655,...",7,2013-02-21 16:19:14.000
97,"[버스, 감성, 카페]",75625,"이유없이 센치해지고 싶은 그런 날, 당신에게 들려주고 픈 노래에요","[665454, 584448, 238840, 676428, 381056, 24653...",9,2019-03-18 01:19:14.000
98,[발라드],140334,걸그룹 발라드 명곡 모음,"[250533, 543094, 528421, 420435, 25538, 220954...",7,2017-03-17 23:28:25.000


In [6]:
import pandas as pd
total = pd.concat([train, val], axis=0)

In [8]:
from konlpy.tag import Komoran
komoran = Komoran

In [9]:
def get_morpher():
    from konlpy.tag import Komoran
    komoran = Komoran()
    return komoran

def get_title_morph(title, morpher):
    return morpher.pos(title)

In [10]:
komoran = get_morpher()

In [18]:
title_morph = [[y[0] for y in get_title_morph(x, komoran)] for x in total['plylst_title']]

In [23]:
from embedding import get_s2v_model
t2v_model = get_s2v_model(title_morph, 2, 100, 3, 5)

In [96]:
t2v_model.most_similar("휴가")

/home/jky/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('여름휴가', 0.8824412226676941),
 ('휴가철', 0.8466458916664124),
 ('바캉스', 0.8345291018486023),
 ('방학', 0.8271308541297913),
 ('막바지', 0.8023832440376282),
 ('기차', 0.7931613326072693),
 ('여름철', 0.7871999144554138),
 ('계획', 0.7841325998306274),
 ('고속도로', 0.7836302518844604),
 ('나기', 0.7806281447410583)]

In [92]:
from gensim.models.keyedvectors import WordEmbeddingsKeyedVectors
from datetime import datetime
from collections import Counter
import numpy as np

def get_time():
    now = datetime.now()
    return now.isoformat()[:-7]


def get_t2p2v_model(train, val, morpher, w2v_model):
    t2p2v_model = WordEmbeddingsKeyedVectors(100)
    ID = []   
    vec = []
    data = pd.concat([train, val], axis=0)
    for id_, title in zip(data['id'], data['plylst_title']):
        tmp_vec = 0
        title_morph = [y[0] for y in get_title_morph(title, morpher)]
        for token in title_morph:
            try: 
                tmp_vec += w2v_model.wv.get_vector(token)
            except KeyError:
                pass
        if type(tmp_vec)!=int:
            ID.append(str(id_))    
            vec.append(tmp_vec)
    t2p2v_model.add(ID, vec)
    
    file_name = "./manual_emb/t2p2v_mdl_" + get_time() + ".model"
    t2p2v_model.save(file_name)
    
    return t2p2v_model


### token to vector
def get_title_vec(token, t2v_mdl):
    try:
        return t2v_mdl.wv.word_vec(token)
    except:
        return np.array([0.0]*100)
    
### sum of token vector(=playlist vector)
def get_title_2_plylst_vec(token_list, t2v_mdl):
    plylst_vec = None
    for idx_, x in enumerate(token_list):
        if idx_ ==0:
            plylst_vec = get_title_vec(x, t2v_mdl)
        else : 
            plylst_vec =  plylst_vec + get_title_vec(x, t2v_mdl)
    return plylst_vec



### word2vec model loading
def load_t2v_mdl(file_path):
    from gensim.models import Word2Vec 
    return Word2Vec.load(file_path)

def load_p2v_mdl(file_path):
    from gensim.models import Word2Vec, KeyedVectors
    return KeyedVectors.load(file_path, mmap='r')


### MS top20 playlist songlist
def get_MS_song_freq(df, song_meta, plylst_vec, p2v_mdl, topn=20, korean=False):
    MS_plylst = get_most_similar_plylst(plylst_vec, p2v_mdl, topn)
    
    total_songs = [] 
    for plylst in MS_plylst:
        songs = get_songs_from_plylst(df, plylst[0], korean=korean, song_meta=song_meta)
        total_songs += songs
        
    return total_songs

def get_MS_tag_freq(df, plylst_vec, p2v_mdl, topn=20):
    MS_plylst = get_most_similar_plylst(plylst_vec, p2v_mdl, topn)
    
    total_songs = [] 
    for plylst in MS_plylst:
        songs = get_tags_from_plylst(df, plylst[0])
        total_songs += songs
        
    return total_songs


### plylist to similar plylist id
def get_most_similar_plylst(plylst_vec, p2v_mdl, topn = 20):
    return p2v_mdl.similar_by_vector(plylst_vec, topn=topn, restrict_vocab=None)

### plylist id to songs
def get_songs_from_plylst(df, id_, korean=False, song_meta=None):
    if korean == False:
        return list(df[df["id"]==int(id_)]["songs"])[0]
    else :
        songs = list(df[df["id"]==int(id_)]["songs"])
        return [song_meta[song_meta['id']==x]["song_name"].item() for x in songs[0]]
    
def get_tags_from_plylst(df, id_):
     return list(df[df["id"]==int(id_)]["tags"])[0]
    
    
def get_count_list(token_list):
    count_list = sorted(dict(Counter(token_list)).items(), key=(lambda x :x[1]), reverse=True)
    return [x for x in count_list if x[1]>1]

In [81]:
total_df = pd.concat([train, val], axis=0)
total_df = total_df.reset_index()
t2v_file_path = "./manual_emb/t2v_mdl.model"
p2v_file_path = "./manual_emb/t2p2v_mdl_2020-07-12T16:34:48.model"
morpher = Komoran()

def t2p2v_rec_model(title, tags_input, songs_input):
    
    song_count_list = []
    tag_count_list = []
    
    if len(title) !=0:
        
        tmp_list = [y[0] for y in get_title_morph(title, morpher)]
        
        t2v_mdl =load_t2v_mdl(t2v_file_path)
        plylst_vec = get_title_2_plylst_vec(tmp_list, t2v_mdl)

        p2v_mdl = load_p2v_mdl(p2v_file_path)

        song_list = get_MS_song_freq(total_df, song_meta, plylst_vec, p2v_mdl, korean= False, topn=200)
        song_count_list = get_count_list(song_list)

        tag_list = get_MS_tag_freq(total_df, plylst_vec, p2v_mdl, topn=200)
        tag_count_list = get_count_list(tag_list)  
        
        add_songs = max(0, 100-len(songs_input))
        add_tags = max(0, 10-len(tags_input))
        
        if len(song_count_list) <= add_songs:
            pass
        else:
            song_count_list = song_count_list[:add_songs]
            
        song_count_list = [x[0] for x in song_count_list]
        song_count_list += songs_input
        
        if len(tag_count_list) <= add_tags:
            pass
        else:
            tag_count_list = tag_count_list[:add_tags]
            
        tag_count_list = [x[0] for x in tag_count_list]
        tag_count_list += tags_input


    else:
        print("empty tags and songs")
    
    return [song_count_list, tag_count_list]

In [73]:
tmp = val[8:10]

In [74]:
tmp

,tags,id,plylst_title,songs,like_cnt,updt_date
8,[스트레스],80810,리듬타면서 빡시게 운동하자!!!(스트레스 날리자):},[],127,2017-02-09 17:33:45.000
9,[],142007,기분 좋은 재즈와 함께 만드는 달달한 하루,[],0,2015-06-22 09:11:02.000


In [94]:
#tmp["songs_base_result"],  tmp["tags_base_result"]= tmp.apply(lambda x : base_model(x["tags"], x["songs"]), axis=1)
val_result = val.apply(lambda x : t2p2v_rec_model(x["plylst_title"], x["tags"], x['songs']), axis=1)

empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags an

In [95]:
val["tags_result"] = [x[1] for x in val_result]
val["songs_result"] = [x[0] for x in val_result]

val[["id", "tags_result", "songs_result"]].to_csv("t2p2v_rec_model_result.csv", encoding="utf-8")

In [34]:
t2p2v_model = get_t2p2v_model(train, val, komoran, t2v_model)

In [36]:
t2p2v_model.most_similar("98472")

[('137778', 0.9838548302650452),
 ('135488', 0.9838548302650452),
 ('21497', 0.9406653046607971),
 ('43785', 0.938018262386322),
 ('625', 0.938018262386322),
 ('41927', 0.9356130361557007),
 ('11064', 0.9256645441055298),
 ('77110', 0.915657639503479),
 ('57556', 0.9152759313583374),
 ('132034', 0.9139619469642639)]

In [49]:
get_plylst_vec(["오늘", "이별"], t2v_model)

array([ 1.4262930e-02, -7.1537817e-01,  1.9424170e-01, -6.0442638e-01,
        3.9980197e-01, -5.5784130e-01, -3.9083162e-01, -1.7139297e+00,
       -8.7656033e-01, -5.5867672e-01,  1.8468983e+00,  1.4170601e+00,
       -2.8478551e-01,  1.4009356e-01, -6.0289896e-01,  3.2642484e-04,
        3.5222366e-01,  7.4016380e-01,  3.6827549e-02,  4.1576576e-01,
       -2.6199540e-01,  1.1177068e+00,  3.0440351e-01, -7.0728588e-01,
        1.4234791e+00,  2.7775729e-01,  5.8506507e-01,  8.4863591e-01,
       -1.0275800e+00, -3.8436413e-02, -2.0280278e-01, -2.5570810e-02,
        8.8938937e-02, -4.8535126e-01,  9.4772744e-01, -1.7840729e+00,
       -2.3381586e-01,  1.0031682e+00, -3.0049196e-01,  4.7818995e-01,
        3.3736941e-01, -6.2925804e-01,  3.7114125e-01,  8.7305284e-01,
        1.4406080e+00, -4.1139746e-01,  3.0430496e-02, -5.1083761e-01,
       -7.8349817e-01, -8.3371866e-01,  1.2252310e-01, -7.1105018e-02,
       -5.8915079e-01, -5.7077557e-01, -6.7398041e-02, -9.4792354e-01,
      

In [37]:
total[total["id"]==98472]

,tags,id,plylst_title,songs,like_cnt,updt_date
94081,[발라드],98472,▶오늘 헤어졌어요,"[591602, 402387, 682279, 323916, 432810, 21902...",17,2010-09-18 09:44:38.000


In [43]:
total[total["id"]==41927]

,tags,id,plylst_title,songs,like_cnt,updt_date
55149,"[슬픔, 추억, 이별, 회상]",41927,오늘헤어졌어요. 가슴아픈 이별노래,"[501713, 193232, 519403, 281936, 116573, 18573...",35,2019-07-09 08:55:53.000


In [9]:
s2v_model = get_s2v_model(total, 2, 100, 3, 5)

In [10]:
p2v_model = get_p2v_model(train, val, s2v_model)

In [14]:
from gensim.models import Word2Vec 
s2v_model = Word2Vec.load("./manual_emb/s2v_mdl_2020-07-08T21:17:03.model")

In [17]:
from gensim.models import Word2Vec, KeyedVectors
p2v_model = KeyedVectors.load("./manual_emb/p2v_mdl_2020-07-08T21:17:26.model", mmap='r')

In [18]:
p2v_model.most_similar("6362")

[('17095', 0.9961541891098022),
 ('98472', 0.9928523302078247),
 ('86701', 0.9927220344543457),
 ('63269', 0.9918633699417114),
 ('84156', 0.9918058514595032),
 ('79909', 0.9914500713348389),
 ('102725', 0.9913939833641052),
 ('5960', 0.9911588430404663),
 ('96197', 0.9911543130874634),
 ('68932', 0.9907352924346924)]

In [236]:
tags_input = ["추억", "회상"]
songs_input = [394031, 394031] 

In [57]:
tmp_list = get_input_df(tags_input, songs_input)

In [249]:
# get_input_df
import numpy as np
from collections import Counter

### input setting
def get_input_df(tags, songs):
    return tags + [str(x) for x in songs]

### word2vec model loading
def get_s2v_mdl(file_path):
    from gensim.models import Word2Vec 
    return Word2Vec.load(file_path)

def get_p2v_mdl(file_path):
    from gensim.models import Word2Vec, KeyedVectors
    return KeyedVectors.load(file_path, mmap='r')

### token to vector
def get_song_tag_vec(token, s2v_mdl):
    try:
        return s2v_mdl.wv.word_vec(token)
    except:
        return np.array([0.0]*100)
    
### sum of token vector(=playlist vector)
def get_plylst_vec(token_list, s2v_mdl):
    plylst_vec = None
    for idx_, x in enumerate(token_list):
        if idx_ ==0:
            plylst_vec = get_song_tag_vec(x, s2v_mdl)
        else : 
            plylst_vec =  plylst_vec + get_song_tag_vec(x, s2v_mdl)
    return plylst_vec

### plylist to similar plylist id
def get_most_similar_plylst(plylst_vec, p2v_mdl, topn = 20):
    return p2v_mdl.similar_by_vector(plylst_vec, topn=topn, restrict_vocab=None)

### plylist id to songs
def get_songs_from_plylst(df, id_, korean=False, song_meta=None):
    if korean == False:
        return list(df[df["id"]==int(id_)]["songs"])[0]
    else :
        songs = list(df[df["id"]==int(id_)]["songs"])
        return [song_meta[song_meta['id']==x]["song_name"].item() for x in songs[0]]
    
def get_tags_from_plylst(df, id_):
     return list(df[df["id"]==int(id_)]["tags"])[0]

### MS top20 playlist songlist
def get_MS_song_freq(df, song_meta, plylst_vec, p2v_mdl, topn=20, korean=False):
    MS_plylst = get_most_similar_plylst(plylst_vec, p2v_mdl, topn)
    
    total_songs = [] 
    for plylst in MS_plylst:
        songs = get_songs_from_plylst(df, plylst[0], korean=korean, song_meta=song_meta)
        total_songs += songs
        
    return total_songs

def get_MS_tag_freq(df, plylst_vec, p2v_mdl, topn=20):
    MS_plylst = get_most_similar_plylst(plylst_vec, p2v_mdl, topn)
    
    total_songs = [] 
    for plylst in MS_plylst:
        songs = get_tags_from_plylst(df, plylst[0])
        total_songs += songs
        
    return total_songs

def get_count_list(token_list):
    count_list = sorted(dict(Counter(token_list)).items(), key=(lambda x :x[1]), reverse=True)
    return [x for x in count_list if x[1]>1]

In [ ]:
s2v_file_path = "./manual_emb/s2v_mdl_2020-07-08T21:17:03.model"
p2v_file_path = "./manual_emb/p2v_mdl_2020-07-08T21:17:26.model"

In [283]:
total_df = pd.concat([train, val], axis=0)
total_df = total_df.reset_index()

def base_model(tags_input, songs_input):

    song_count_list = []
    tag_count_list = []
    
    if len(tags_input + songs_input) !=0:

        tmp_list = get_input_df(tags_input, songs_input)

        s2v_mdl =get_s2v_mdl(s2v_file_path)
        plylst_vec = get_plylst_vec(tmp_list, s2v_mdl)

        p2v_mdl = get_p2v_mdl(p2v_file_path)

        song_list = get_MS_song_freq(total_df, song_meta, plylst_vec, p2v_mdl, korean= False, topn=200)
        song_count_list = get_count_list(song_list)

        tag_list = get_MS_tag_freq(total_df, plylst_vec, p2v_mdl, topn=200)
        tag_count_list = get_count_list(tag_list)  
        
        add_songs = max(0, 100-len(songs_input))
        add_tags = max(0, 10-len(tags_input))
        
        if len(song_count_list) <= add_songs:
            pass
        else:
            song_count_list = song_count_list[:add_songs]
            
        song_count_list = [x[0] for x in song_count_list]
        song_count_list += songs_input
        
        if len(tag_count_list) <= add_tags:
            pass
        else:
            tag_count_list = tag_count_list[:add_tags]
            
        tag_count_list = [x[0] for x in tag_count_list]
        tag_count_list += tags_input


    else:
        print("empty tags and songs")
    
    return [song_count_list, tag_count_list]

In [289]:
#tmp["songs_base_result"],  tmp["tags_base_result"]= tmp.apply(lambda x : base_model(x["tags"], x["songs"]), axis=1)
val_result = val.apply(lambda x : base_model(x["tags"], x["songs"]), axis=1)

empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags and songs
empty tags an

In [290]:
val["tags_result"] = [x[1] for x in val_result] 
val["songs_result"] = [x[0] for x in val_result] 

In [292]:
val[["id", "tags_result", "songs_result"]].to_csv("base_list.csv", encoding="utf-8")

In [299]:
song_meta.tail()

,song_gn_dtl_gnr_basket,issue_date,album_name,album_id,artist_id_basket,song_name,song_gn_gnr_basket,artist_name_basket,id
707984,[GN2001],19991219,The Best Best Of The Black President,65254,[166499],Coffin For Head Of State,[GN2000],[Fela Kuti],707984
707985,[GN0901],19860000,True Colors,44141,[11837],Change Of Heart,[GN0900],[Cyndi Lauper],707985
707986,"[GN0105, GN0101]",20160120,행보 2015 윤종신 / 작사가 윤종신 Live Part.1,2662866,[437],스치듯 안녕,[GN0100],[윤종신],707986
707987,"[GN1807, GN1801]",20131217,명상의 시간을 위한 뉴에이지 음악,2221722,[729868],숲의 빛,[GN1800],[Nature Piano],707987
707988,"[GN0601, GN0604]",19980000,김경호 Live,34663,[895],Queen 명곡 멜로디,[GN0600],[김경호],707988


In [294]:
len(song_meta)

707989

In [295]:
from konlpy.tag import Komoran

In [296]:
komoran =  Komoran()

In [298]:
komoran.pos("Bach : Partita No. 4 In D Major, BWV 828")

[('Bach', 'SL'),
 (':', 'SP'),
 ('Partita', 'SL'),
 ('No', 'SL'),
 ('.', 'SF'),
 ('4', 'SN'),
 ('In', 'SL'),
 ('D', 'SL'),
 ('Major', 'SL'),
 (',', 'SP'),
 ('BWV', 'SL'),
 ('828', 'SN')]

In [104]:
from collections import Counter
result_dict = dict(Counter(songs_total))
sorted(result_dict.items(), key=(lambda x :x[1]), reverse=True)



[(116847, 14),
 (284814, 9),
 (424869, 8),
 (177284, 8),
 (201145, 8),
 (582905, 8),
 (669617, 8),
 (143652, 7),
 (356571, 7),
 (397924, 7),
 (485957, 7),
 (149779, 7),
 (4412, 7),
 (555134, 6),
 (591047, 6),
 (367459, 6),
 (347303, 6),
 (433447, 6),
 (21865, 6),
 (438086, 6),
 (237282, 6),
 (380859, 6),
 (87752, 5),
 (633203, 5),
 (70662, 5),
 (158554, 5),
 (298646, 5),
 (262145, 5),
 (592386, 5),
 (16847, 5),
 (68266, 4),
 (199390, 4),
 (411756, 4),
 (486220, 4),
 (599290, 4),
 (569548, 4),
 (477964, 4),
 (133330, 4),
 (4083, 4),
 (426013, 4),
 (268811, 4),
 (256465, 4),
 (629177, 4),
 (466048, 4),
 (186358, 4),
 (96825, 4),
 (651339, 4),
 (665144, 4),
 (518239, 3),
 (548735, 3),
 (11914, 3),
 (170578, 3),
 (477689, 3),
 (587193, 3),
 (554558, 3),
 (122384, 3),
 (32042, 3),
 (415774, 3),
 (570100, 3),
 (173709, 3),
 (705462, 3),
 (68488, 3),
 (402661, 3),
 (635160, 3),
 (583525, 3),
 (69933, 3),
 (488423, 3),
 (531732, 3),
 (38311, 3),
 (15958, 3),
 (661213, 3),
 (221986, 3),
 (45421

In [137]:
unused_song = []
new_dic = {}
for i in result_dict.items():
    if str(i[0]) not in songs and i[1]>=2:
        print(str(i[0]))
        new_dic[str(i[0])] = i[1]
        
print(sorted(new_dic.items(), key=(lambda x :x[1]), reverse=True))

11914
170578
411756
486220
367459
70662
16083
38581
477689
347303
590379
284814
599290
23570
177284
569548
480590
433447
230729
356571
158554
201145
21865
397924
153734
587193
285445
675126
554558
298646
477964
122384
547977
582905
262145
143877
188590
133330
319297
592386
122998
32042
142498
438086
579395
4083
415774
395858
669617
237282
485957
1099
8187
699365
570100
638346
173709
622190
149779
705462
380859
68488
4412
546064
402661
426013
635160
583525
274820
69933
485142
444479
488423
184540
268811
531732
256465
292655
317710
192195
94820
21408
320382
429256
127614
695494
629177
466048
16847
128242
38311
65958
15958
661213
186358
221986
543556
454218
414755
134728
134583
46284
589242
96825
28024
132689
653071
651339
38836
349886
675797
677063
85800
688951
665144
602629
539892
524168
374617
63874
503911
355713
235609
678294
565
220781
77781
679405
184015
169935
635759
135756
527887
65062
106438
466004
566376
50055
325001
93307
322776
436255
344327
439190
413422
18768
25892
157566
54

In [130]:
new_dic

[]

In [121]:
len(unused_song)

482

In [123]:
len(result_dict.items())

498

In [122]:
len(songs)

16

In [22]:

def get_p2v_array(input_data):
    total_array = None

    for idx_, token in enumerate(input_data):

        if token in s2v_model.wv.vocab:

            if idx_==0:
                total_array = s2v_model.wv.word_vec(token)
            else:
                total_array = total_array + s2v_model.wv.word_vec(token)

        else:
            pass
        
    return total_array